# CAB420, Practical 1 - Question 1 Solution

## Combining and Filtering Multiple Datasets

CAB420_Prac1.zip contains a number of datasets, split into two directories as follows:
* BOM, which contains Bureau of Meteorology data for Brisbane City from the years 1999-2019. The data is split into three files.
 * IDCJAC0009_040913_1800_Data.csv contains daily rainfall data;
 * IDCJAC0010_040913_1800_Data.csv contains maximum daily temperature data; and
 * IDCJAC0013_040913_1800_Data.csv contains daily solar exposure data.
* BCCCyclewayCounts contains five years data (from 2014-2018) for Brisbane City Council cycleways, with data for each year being in a separate file (i.e. bike-ped-auto-counts-2014.csv contains data for the year 2014).

You are to combine these datasets into a single table using Python (or the programming language of your choice such that):
* You have a single table that spans the time period of the BCCCyclewayCounts data;
* Duplicate information is avoided (i.e. you don’t have multiple date columns, or similar);
* For the cycle way data, only columns that are available in all years data are included in the final table (i.e. if a counter is available in 2014 − 2017, but not 2018, that column should be excluded.

In [1]:
# numpy handles pretty much anything that is a number/vector/matrix/array
import numpy as np
# pandas handles dataframes
import pandas as pd
# matplotlib emulates Matlabs plotting functionality
import matplotlib.pyplot as plt
# stats models is a package that is going to perform the regression analysis
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
# os allows us to manipulate variables on out local machine, such as paths and environment variables
import os
# self explainatory, dates and times
from datetime import datetime, date
# a helper package to help us iterate over objects
import itertools

Create single datetime object from year, month and day

In [2]:
def create_date(row):
    # create string with date it format Y:m:d
  return datetime.strptime('{:04d}/{:02d}/{:02d}'.format(row.Year, row.Month, row.Day),
                           '%Y/%m/%d')

Create a function to load our weather data.

In [3]:
def load_bom(root_dir):
  """load in the BOM data and concatenate it all"""
  rain = pd.read_csv(os.path.join(root_dir, 'IDCJAC0009_040913_1800_Data.csv'))
  temp = pd.read_csv(os.path.join(root_dir, 'IDCJAC0010_040913_1800_Data.csv'))
  solar = pd.read_csv(os.path.join(root_dir, 'IDCJAC0016_040913_1800_Data.csv'))
  
  # want all the date ranges to be the same
  rain = rain[(rain.Year >= 2014) & (rain.Year <= 2018)]
  temp = temp[(temp.Year >= 2014) & (temp.Year <= 2018)]
  solar = solar[(solar.Year >= 2014) & (solar.Year <= 2018)]
  # create a datetime object for each row, and store it in the Date column
  rain['Date'] = rain.apply(create_date, axis=1)
  temp['Date'] = temp.apply(create_date, axis=1)
  solar['Date'] = solar.apply(create_date, axis=1)
  # now merge them all based on the date, so will create a single Date
  # column that has them all in
  bom_combined = pd.merge(rain, temp, on='Date')
  bom_combined = pd.merge(bom_combined, solar, on='Date')
  return bom_combined

Load our weather data and have a look at the resulting table.

In [4]:
bom = load_bom('../../data/BrisbaneData/BOM/')
bom.head()

,Product code_x,Bureau of Meteorology station number_x,Year_x,Month_x,Day_x,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality_x,Date,Product code_y,...,Day_y,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality_y,Product code,Bureau of Meteorology station number,Year,Month,Day,Daily global solar exposure (MJ/m*m)
0,IDCJAC0009,40913,2014,1,1,0.0,1.0,N,2014-01-01,IDCJAC0010,...,1,30.6,1.0,Y,IDCJAC0016,40913,2014,1,1,31.2
1,IDCJAC0009,40913,2014,1,2,0.0,1.0,N,2014-01-02,IDCJAC0010,...,2,31.8,1.0,Y,IDCJAC0016,40913,2014,1,2,23.4
2,IDCJAC0009,40913,2014,1,3,1.0,1.0,N,2014-01-03,IDCJAC0010,...,3,34.5,1.0,Y,IDCJAC0016,40913,2014,1,3,29.6
3,IDCJAC0009,40913,2014,1,4,0.0,1.0,N,2014-01-04,IDCJAC0010,...,4,38.7,1.0,Y,IDCJAC0016,40913,2014,1,4,30.5
4,IDCJAC0009,40913,2014,1,5,0.0,1.0,N,2014-01-05,IDCJAC0010,...,5,33.6,1.0,Y,IDCJAC0016,40913,2014,1,5,15.7


looks like there are some repeated columns (from the year/month/day), lets get rid of those

In [5]:
print(bom.columns.values)

['Product code_x' 'Bureau of Meteorology station number_x' 'Year_x'
 'Month_x' 'Day_x' 'Rainfall amount (millimetres)'
 'Period over which rainfall was measured (days)' 'Quality_x' 'Date'
 'Product code_y' 'Bureau of Meteorology station number_y' 'Year_y'
 'Month_y' 'Day_y' 'Maximum temperature (Degree C)'
 'Days of accumulation of maximum temperature' 'Quality_y' 'Product code'
 'Bureau of Meteorology station number' 'Year' 'Month' 'Day'
 'Daily global solar exposure (MJ/m*m)']


In [6]:
bom = bom.drop(['Year', 'Month', 'Day',
                'Year_x', 'Month_x', 'Day_x',
                'Year_y', 'Month_y', 'Day_y'], axis=1)
bom.head()

,Product code_x,Bureau of Meteorology station number_x,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality_x,Date,Product code_y,Bureau of Meteorology station number_y,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality_y,Product code,Bureau of Meteorology station number,Daily global solar exposure (MJ/m*m)
0,IDCJAC0009,40913,0.0,1.0,N,2014-01-01,IDCJAC0010,40913,30.6,1.0,Y,IDCJAC0016,40913,31.2
1,IDCJAC0009,40913,0.0,1.0,N,2014-01-02,IDCJAC0010,40913,31.8,1.0,Y,IDCJAC0016,40913,23.4
2,IDCJAC0009,40913,1.0,1.0,N,2014-01-03,IDCJAC0010,40913,34.5,1.0,Y,IDCJAC0016,40913,29.6
3,IDCJAC0009,40913,0.0,1.0,N,2014-01-04,IDCJAC0010,40913,38.7,1.0,Y,IDCJAC0016,40913,30.5
4,IDCJAC0009,40913,0.0,1.0,N,2014-01-05,IDCJAC0010,40913,33.6,1.0,Y,IDCJAC0016,40913,15.7


Now we want to load in the BCC data and see which columns are available in all of the different data sets. Once again, we'll create a function to do that.

In [7]:
def load_bcc(root_dir):
  bcc_2014 = pd.read_csv(os.path.join(root_dir, 'bike-ped-auto-counts-2014.csv'))
  bcc_2015 = pd.read_csv(os.path.join(root_dir, 'bike-ped-auto-counts-2015.csv'))
  bcc_2016 = pd.read_csv(os.path.join(root_dir, 'bike-ped-auto-counts-2016.csv'))
  bcc_2017 = pd.read_csv(os.path.join(root_dir, 'bike-ped-auto-counts-2017.csv'))
  bcc_2018 = pd.read_csv(os.path.join(root_dir, 'bike-ped-auto-counts-2018.csv'))
  # make sure the date is the correct format
  # this is important, don't assume that Python or pandas will determine the
  # format correctly for you (talking from experience, learn from my mistakes)
  bcc_2014.Date = pd.to_datetime(bcc_2014.Date, format='%d/%m/%Y')
  bcc_2015.Date = pd.to_datetime(bcc_2015.Date, format='%d/%m/%Y')
  bcc_2016.Date = pd.to_datetime(bcc_2016.Date, format='%d/%m/%Y')
  bcc_2017.Date = pd.to_datetime(bcc_2017.Date, format='%d/%m/%Y')
  bcc_2018.Date = pd.to_datetime(bcc_2018.Date, format='%d/%m/%Y')
  # lets get the column values, and see which are consistent across them all
  # itertools is a helper module to help us convert the list of dataframe column
  # values (a list of lists) to one big flattened out list.
  # This flattened lists could have repeated entries
  all_columns_repeated = list(itertools.chain(*(
    bcc_2014.columns.values, bcc_2015.columns.values,
    bcc_2016.columns.values, bcc_2017.columns.values,
    bcc_2018.columns.values)))
  # creating a set of these column values  will remove duplicates
  all_columns = set(all_columns_repeated)
  # now lets perform the intersection of this set with each set of the column
  # values from each data set.
  # this will tell us which columns are common to every data set
  common_column = all_columns.intersection(bcc_2014.columns.values, bcc_2015.columns.values,
                                           bcc_2016.columns.values, bcc_2017.columns.values,
                                           bcc_2018.columns.values)
  # now lets get just the common columns from each data frame, and then
  # concatenate them all
  bcc_2014 = bcc_2014[common_column]
  bcc_2015 = bcc_2015[common_column]
  bcc_2016 = bcc_2016[common_column]
  bcc_2017 = bcc_2017[common_column]
  bcc_2018 = bcc_2018[common_column]
  # now concatenate them all (concat vertically)
  bcc = pd.concat([bcc_2014, bcc_2015, bcc_2016,
                   bcc_2017, bcc_2018])
  return bcc

Load the BCC data and print the top of the table.

In [8]:
bcc = load_bcc('../../data/BrisbaneData/BCCCyclewayCounts/')
bcc.head()

,Story Bridge West Pedestrian Inbound,Granville Street Bridge Cyclists Inbound,Kedron Brook Bikeway Lutwyche Pedestrians Inbound,Bicentennial Bikeway Cyclists Inbound,Jack Pesch Bridge Cyclists Outbound,Story Bridge East Pedestrian Outbound,Story Bridge West Cyclists Inbound,Riverwalk Cyclists Outbound,Jack Pesch Bridge Pedestrians Inbound,Story Bridge East Pedestrian Inbound,...,Kedron Brook Bikeway Lutwyche Cyclists Outbound,Kedron Brook Bikeway Mitchelton Cyclist Inbound,Ekibin Park Cyclists Outbound,Jack Pesch Bridge Cyclists Inbound,Kedron Brook Bikeway Mitchelton Cyclist Outbound,Story Bridge East Cyclists Inbound,Bishop Street Cyclists Outbound,Story Bridge East Cyclists Outbound,Kedron Brook Bikeway Mitchelton Pedestrian Inbound,Bicentennial Bikeway Cyclists Outbound
0,0.0,NaN,NaN,569.0,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,572.0
1,0.0,NaN,NaN,658.0,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,700.0
2,0.0,NaN,NaN,463.0,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,412.0
3,0.0,NaN,NaN,410.0,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,364.0
4,0.0,NaN,NaN,654.0,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,641.0


Now we can combine (merge) both the BOM and BCC dataframe. Again, want to merge using the date to make sure eveything is aligned and organised properly. First, we want to convert the Date column to a format consistent with the BOM data.

In [9]:
bcc.Date = pd.to_datetime(bcc.Date)
combined = pd.merge(bom, bcc, on='Date')

We now have a merged dataset. 

In [10]:
combined.head()

,Product code_x,Bureau of Meteorology station number_x,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality_x,Date,Product code_y,Bureau of Meteorology station number_y,Maximum temperature (Degree C),Days of accumulation of maximum temperature,...,Kedron Brook Bikeway Lutwyche Cyclists Outbound,Kedron Brook Bikeway Mitchelton Cyclist Inbound,Ekibin Park Cyclists Outbound,Jack Pesch Bridge Cyclists Inbound,Kedron Brook Bikeway Mitchelton Cyclist Outbound,Story Bridge East Cyclists Inbound,Bishop Street Cyclists Outbound,Story Bridge East Cyclists Outbound,Kedron Brook Bikeway Mitchelton Pedestrian Inbound,Bicentennial Bikeway Cyclists Outbound
0,IDCJAC0009,40913,0.0,1.0,N,2014-01-01,IDCJAC0010,40913,30.6,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,572.0
1,IDCJAC0009,40913,0.0,1.0,N,2014-01-02,IDCJAC0010,40913,31.8,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,700.0
2,IDCJAC0009,40913,1.0,1.0,N,2014-01-03,IDCJAC0010,40913,34.5,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,412.0
3,IDCJAC0009,40913,0.0,1.0,N,2014-01-04,IDCJAC0010,40913,38.7,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,364.0
4,IDCJAC0009,40913,0.0,1.0,N,2014-01-05,IDCJAC0010,40913,33.6,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,641.0


We have a few columns, particularly from the weather data, that we're never going to use, so let's remove them too.

In [11]:
to_drop = ['Product code_x', 'Bureau of Meteorology station number_x',
           'Product code_y', 'Bureau of Meteorology station number_y',
           'Product code', 'Bureau of Meteorology station number',
           'Period over which rainfall was measured (days)',
           'Days of accumulation of maximum temperature',
           'Quality_x', 'Quality_y']
combined = combined.drop(to_drop, axis=1)
combined.head()

,Rainfall amount (millimetres),Date,Maximum temperature (Degree C),Daily global solar exposure (MJ/m*m),Story Bridge West Pedestrian Inbound,Granville Street Bridge Cyclists Inbound,Kedron Brook Bikeway Lutwyche Pedestrians Inbound,Bicentennial Bikeway Cyclists Inbound,Jack Pesch Bridge Cyclists Outbound,Story Bridge East Pedestrian Outbound,...,Kedron Brook Bikeway Lutwyche Cyclists Outbound,Kedron Brook Bikeway Mitchelton Cyclist Inbound,Ekibin Park Cyclists Outbound,Jack Pesch Bridge Cyclists Inbound,Kedron Brook Bikeway Mitchelton Cyclist Outbound,Story Bridge East Cyclists Inbound,Bishop Street Cyclists Outbound,Story Bridge East Cyclists Outbound,Kedron Brook Bikeway Mitchelton Pedestrian Inbound,Bicentennial Bikeway Cyclists Outbound
0,0.0,2014-01-01,30.6,31.2,0.0,NaN,NaN,569.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,572.0
1,0.0,2014-01-02,31.8,23.4,0.0,NaN,NaN,658.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,700.0
2,1.0,2014-01-03,34.5,29.6,0.0,NaN,NaN,463.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,412.0
3,0.0,2014-01-04,38.7,30.5,0.0,NaN,NaN,410.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,364.0
4,0.0,2014-01-05,33.6,15.7,0.0,NaN,NaN,654.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,641.0


Finally, save the dataset.

In [12]:
combined.to_csv('combinedData-Python.csv')